In [208]:
%esptool erase



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7f76341540b8, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Executing:
  esptool.py --port /dev/ttyUSB3 erase_flash

esptool.py v2.0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 2.4s
Hard resetting...


In [209]:
%esptool esp8266 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

Executing:
  esptool.py --port /dev/ttyUSB3 --baud 460800 write_flash --flash_size=detect -fm dio 0 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

esptool.py v2.0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 460800
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 615876 bytes to 401882...
Wrote 615876 bytes (401882 compressed) at 0x00000000 in 9.2 seconds (effective 534.0 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting...


In [226]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [215]:
from mqtt_as import config, MQTTClient
import uasyncio as asyncio
import time
from machine import Pin, PWM, I2C

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)
  

.

In [228]:
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
print(list(map(hex, i2c.scan())))


['0x76', '0x77']


In [181]:
print(i2c.readfrom_mem(0x77, 0xD0, 1))

b'`'


In [212]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
boardname    esp8266sens2

Sent 5 lines (108 bytes) to config.txt.


In [218]:
%sendtofile --source ../Sensor_Kennel/BME280_funcs.py

Sent 79 lines (3304 bytes) to BME280_funcs.py.


In [219]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [220]:
%sendtofile main.py

import time
from machine import Pin, PWM, I2C

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)
    
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
print(list(map(hex, i2c.scan())))

from mqtt_as import config, MQTTClient
import uasyncio as asyncio
from BME280_funcs import bme280init, readBME280


config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]

boardname = fconfig["boardname"].encode()    

tbme280A = boardname+b'/bme280/A'
tbme280B = boardname+b'/bme280/B'

client = MQTTClient(config)

async def ledtask():
    while True:
        pinled.value(0)
        await asyncio.sleep_ms(20 if client.isconnected() else 600)
        pinled.value(1)
        await asyncio.sleep_ms(2000)
        
async def bme280task(addr, topic):
    btoinit = True
    while True:
        try:
            if btoinit:
                bme280init(i2c, addr)
                btoinit = False
                await asyncio.sleep_ms(100)
            meas = readBME280(addr)
            print(topic, meas)
            payload = " ".join(map(str, meas))
            await client.publish(topic, payload)
        except OSError as e:
            print("bme280task", e)
            btoinit = True
        await asyncio.sleep_ms(500)
        
async def connectingtask():
    await client.connect()
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+b'/ip', ipnumber, retain=True, qos=1)
        
aloop = asyncio.get_event_loop()
aloop.create_task(bme280task(0x76, boardname+b'/bme280/A'))
aloop.create_task(bme280task(0x77, boardname+b'/bme280/B'))
aloop.create_task(ledtask())
aloop.create_task(connectingtask())
aloop.run_forever()



Sent 66 lines (1911 bytes) to main.py.


In [201]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
[Timed out waiting for recognizable response]
Disconnecting [paste mode not working]
  (You may need to reset the device)

In [190]:
fconfig = dict(x.split()  for x in open("config.txt"))
import network
si = network.WLAN()
si.active(True)
si.connect(fconfig["wifiname"], fconfig["wifipassword"])
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print(ipnumber)



scandone
192.168.43.71


In [191]:
from umqtt.robust import MQTTClient
client = MQTTClient(fconfig["boardname"], fconfig["mqttbroker"])
client.connect()


In [192]:
client.publish("esp32temp/DS18B20/0", "hi there")